In [ ]:
# # Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install -e .

In [1]:
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import torch
from torch import nn
import torchvision.transforms as transforms

import pytorch_lightning as pl
from torchmetrics.classification import BinaryAccuracy
from pytorch_lightning.callbacks import ModelCheckpoint 

from src.data.project1.dataloader import get_loaders, get_normalization_constants
from src.utils import set_seed

In [2]:
SEED = 0

set_seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
accuracy = BinaryAccuracy().to(device)

## Loading model

In [3]:
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=2)

## Dataloader

In [4]:
ROOT = '/dtu/datasets1/02514/hotdog_nothotdog'
SEED = 0
BATCH_SIZE = 32

# Get normalization constants
train_mean, train_std = get_normalization_constants(root=ROOT, seed=SEED)

# Define transforms for training
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),         # flips "left-right"
    # transforms.RandomVerticalFlip(p=1.0),           # flips "upside-down"
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(60, 70)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=train_mean, 
        std=train_std, 
    )
])

# Define transforms for test and validation
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=train_mean, 
        std=train_std, 
    )
])

# Get data loaders with applied transformations
loaders = get_loaders(
    root=ROOT, 
    batch_size=BATCH_SIZE, 
    seed=SEED, 
    train_transforms=train_transforms, 
    test_transforms=test_transforms, 
    num_workers=24
)

Computing mean of training split...: 100%|█| 1638/1638 [00:04<00:00, 330.43
Computing std. dev. of training split...: 100%|█| 1638/1638 [00:06<00:00, 2



Mean: tensor([0.5132, 0.4369, 0.3576])
Std. dev.: tensor([0.0214, 0.0208, 0.0223])


## Output of model

In [5]:
# Get example batch
batch, targets = next(iter(loaders['train'])) 
out = model(batch)
torch.argmax(nn.functional.softmax(out, dim=1), dim = 1)

tensor([1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 0, 0, 1])

In [6]:
# torch.nn.functional.one_hot(targets, num_classes=2)

## Model-class (Lightning)

In [36]:
# Transfer learning model
class HotdogEfficientNet(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # Load model
        self.efficient_net = timm.create_model('efficientnet_b4', pretrained=True, num_classes=2)
        # Freeze weights
        for param in self.efficient_net.parameters():
            param.requires_grad = False
        
        # Require gradient for classification layer
        self.efficient_net.classifier.requires_grad_()


        # Define metrics and loss criterion
        self.criterion = nn.BCEWithLogitsLoss() # nn.BCELoss()
        self.accuracy = BinaryAccuracy().to(self.device)

        # Set up logging option
        self.model_checkpoint = ModelCheckpoint(
            monitor = "val_loss",
            verbose=True,
            filename="{epoch}_{val_loss:.4f}"
        )

    def forward(self,x):
        return self.efficient_net(x) # out # torch.argmax(nn.functional.softmax(out, dim=1), dim = 1)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = 1e-4)

    def training_step(self, batch, batch_idx):
        # Extract and process input
        x, y = batch
        y = torch.nn.functional.one_hot(y, num_classes=2) 
        y = y.to(torch.float32)

        # Get prediction, loss and accuracy
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)

        # logs metrics for each training_step - [default:True],
        # the average across the epoch, to the progress bar and logger-[default:False]
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True),
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # Extract and process input
        x, y = batch
        x, y = x.to(self.device), y.to(self.device)
        y = torch.nn.functional.one_hot(y, num_classes=2) 
        y = y.to(torch.float32) 

        # Get prediction, loss and accuracy
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)

        # logs metrics for each validation_step - [default:False]
        #the average across the epoch - [default:True]
        self.log("val_acc", acc, prog_bar=True, logger=True),
        self.log("val_loss", loss, prog_bar=True, logger=True)

## Training

In [37]:
# dm = HotDogDataModule()
HotdogModel = HotdogEfficientNet()

#CPU:default,GPU:gpus,TPU:tpu_cores
trainer = pl.Trainer(
    devices=1, 
    accelerator="gpu", 
    max_epochs = 10,
    log_every_n_steps=2,
    callbacks=[HotdogModel.model_checkpoint]
) 

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [39]:
trainer.fit(
    model=HotdogModel,
    train_dataloaders = loaders['train'],
    val_dataloaders = loaders['validation'], 
) 

#manually you can save best checkpoints - 
trainer.save_checkpoint("DL-COMVIS/models/modelshotdog_efficient_net.ckpt")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name          | Type              | Params
----------------------------------------------------
0 | efficient_net | EfficientNet      | 17.6 M
1 | criterion     | BCEWithLogitsLoss | 0     
2 | accuracy      | BinaryAccuracy    | 0     
----------------------------------------------------
3.6 K     Trainable params
17.5 M    Non-trainable params
17.6 M    Total params
70.209    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/work3/s194253/02514/DL-COMVIS/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/work3/s194253/02514/DL-COMVIS/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/work3/s194253/02514/DL-COMVIS/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` ar

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Epoch 7, global step 416: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 8, global step 468: 'val_loss' reached 1.47394 (best 1.47394), saving model to '/work3/s194253/02514/lightning_logs/version_4/checkpoints/epoch=8_val_loss=1.4739.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

Epoch 9, global step 520: 'val_loss' reached 1.43407 (best 1.43407), saving model to '/work3/s194253/02514/lightning_logs/version_4/checkpoints/epoch=9_val_loss=1.4341.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=10` reached.
